<a href="https://colab.research.google.com/github/mshivam23/Machine_learning/blob/main/Online%20writer%20identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tqdm==4.36.1
!pip install tensorflow_gpu==2.0.0
!pip install tensorflow==2.0.0

In [2]:
!pip install numpy==1.17.3

In [3]:
#importing models and files
import json
from random import choice, sample, randint
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Dense, GlobalMaxPool1D, GlobalAvgPool1D, Concatenate, Multiply, Dropout, \
    Subtract, Bidirectional, MaxPool1D, GRU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.python.ops import math_ops
import tensorflow
tensorflow.keras.metrics.SparseCategoricalAccuracy(name='sparse_categorical_accuracy', dtype=None)
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import AvgPool1D
from sklearn import preprocessing

In [4]:
#for limiting the stoke value length to max size=40, coz strokes are not of equal length
def cap(seq, cap_val=0, max_size=500):   #will fetch the stroke sequence 
    seq = seq[:max_size]                    #will take the sequence till the max size
    seq = seq + [cap_val] * (max_size - len(seq))   #if the length of the stroke is less than 40, it will add zeros in order to limit the lenght to 40
    return seq                           #returning the sequenece

In [5]:
#we would prefer to go for relative features so that model will be insensitive to the absolue position,but it will take the relative positions, so it will only be sensitive to the gap between two samples
def to_delta(x):                                      # for calculating the relative features, 
    x = [0] + x
    deltas = [b - a for a, b in zip(x[:-1], x[1:])]
    return deltas            #returning the relative features

def to_delta2(x):                                      # for calculating the relative features, 
    x = [0] + x
    deltas = [b - a for a, b in zip(x[:-1], x[20:])]
    return deltas            #returning the relative features


In [6]:
def normalize(arr):
    norm_arr = []
    diff = 1
    diff_arr = max(arr) - min(arr)    
    for i in arr:
        temp = (((i - min(arr))*diff)/diff_arr) 
        norm_arr.append(temp)
    return norm_arr

In [15]:
#for generating sample or substrokes
def generate_sample_from_strokes(data, size_max=1300, pen_val=10):
    x = []           #X parameters
    y = []           #Y parameters
    pen = []         #pen up and down information
    for stroke in data['strokes']:               #loop for generating the samples to be fedded 
        x += stroke['x']                    #will keep on adding the x 
        y += stroke['y']                     #will keep on adding the Y
        _pen = [0] * len(stroke['x'])         
        _pen[0] = pen_val
        pen += _pen                       #will keep on adding pen up and down information

    pen = cap(pen, cap_val=0, max_size=size_max)   #cap function will limit the pen info to be limited to max_size=40
    x = cap(x, cap_val=0, max_size=size_max)       #will give the X sub-stroke limited to length=40
    y = cap(y, cap_val=0, max_size=size_max)        # will give the substroke of Y limited to max length=40
    x1 = to_delta(x)                                 #will give the relative position of X
    y1 = to_delta(y)



    return list(zip(x1,y1,pen))                     # will give the stoke having X,Y,pen info as features

In [8]:
#for generating sample or substrokes
def generate_sample_from_strokes(data, size_max=1000, pen_val=10):
    x = []           #X parameters
    y = []           #Y parameters
    pen = []         #pen up and down information
    for stroke in data['strokes']:               #loop for generating the samples to be fedded 
        x += stroke['x']                    #will keep on adding the x 
        y += stroke['y']                     #will keep on adding the Y
        _pen = [0] * len(stroke['x'])         
        _pen[0] = pen_val
        pen += _pen                       #will keep on adding pen up and down information

    pen = cap(pen, cap_val=0, max_size=size_max)   #cap function will limit the pen info to be limited to max_size=40
    x = cap(x, cap_val=0, max_size=size_max)       #will give the X sub-stroke limited to length=40
    y = cap(y, cap_val=0, max_size=size_max)        # will give the substroke of Y limited to max length=40
    x1 = to_delta(x)                                 #will give the relative position of X
    y1 = to_delta(y)
    x2= to_delta2(x)
    y2=to_delta2(y)



    return list(zip(x1,y1,x2,y2,pen))                     # will give the stoke having X,Y,pen info as features

In [9]:
def read_sample(path):                #for reding the data of strokes from files from json files
    with open(path, "r") as f:            
        data = json.load(f)                                  #loading data from json files
    return generate_sample_from_strokes(data)                #generating samples from the strokes of the data

from itertools import islice
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [10]:
def gen(writers_map, batch_size=128):
  writers = list(writers_map.keys())
  while True:
    labels = [randint(0, 49) for _ in range(128)]

    X = [choice(writers_map[writers[index]]) for index in labels]
    X = np.array([read_sample(x) for x in X])/10
    yield X, labels

In [11]:
def encoder(seq_dim=3):
    in_seq_input = Input((None, seq_dim), name='in_sequence')
    in_seq = Bidirectional(LSTM(10, return_sequences=True), name="gru_1")(in_seq_input)
    in_seq = MaxPool1D(pool_size=10)(in_seq)
    in_seq = Bidirectional(LSTM(25, return_sequences=True), name="gru_2")(in_seq_input)
    in_seq = MaxPool1D(pool_size=10)(in_seq)
    in_seq = Bidirectional(LSTM(50, return_sequences=True), name="gru_3")(in_seq_input)
    model = Model(in_seq_input, in_seq)
    return model

In [12]:
def baseline_model(seq_dim=3):
    input_1 = Input(shape=(None, 3))     #defining input for the model
    base_model = encoder(seq_dim=3)    # for passing the input from the model
    x1 = base_model(input_1)      #getting first input as delta X  
    x1 = Concatenate(axis=-1)([GlobalMaxPool1D()(x1), GlobalAvgPool1D()(x1)])     
    x = Dense(1024, activation="relu")(x1)# can put any at 100 place, can try with other number as well
    x = Dropout(0.5)(x)
    out =Dense(50, activation="softmax")(x)
    model = Model([input_1], out)
    model.compile(loss="sparse_categorical_crossentropy", metrics=['SparseCategoricalAccuracy'], optimizer=Adam(0.001))
    model.summary()
    return model

In [13]:
from google.colab import drive
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [ ]:
if __name__ == "__main__":
    import random
    from itertools import islice

    with open('/content/gdrive/MyDrive/project/p/writer_json_mapping.json', 'r') as f:
        writer_json_mapping = json.load(f) # loading json file (contains mapping writert to json strokes files)
    indexes=take(50,writer_json_mapping)
    Dict={}
    t=0
    for i in indexes:
      if i in writer_json_mapping.keys():
        Dict[t]=writer_json_mapping[i]
        t=t+1
    

    train_writers_mapping = {}
    val_writers_mapping = {}

    for k, v in Dict.items():
      train = []
      val = []

      for writing in v:
          if random.random() < 0.75 or not train:
              train.append(writing)
          else:
              val.append(writing)

      train_writers_mapping[k] = train
      val_writers_mapping[k] = val
      if not val:
          val_writers_mapping[k] = train

    file_path = "/content/gdrive/MyDrive/project/handwriting_online_LSTM5.h5"

    checkpoint = ModelCheckpoint(file_path, monitor='val_SparseCategoricalAccuracy', verbose=1, save_best_only=True, mode='max')

    reduce_on_plateau = ReduceLROnPlateau(monitor="val_SparseCategoricalAccuracy", mode="max", factor=0.01, patience=2, verbose=1)

    callbacks_list = [checkpoint, reduce_on_plateau]

    model = baseline_model()

    try:
        model.load_weights(file_path)
    except:
        print("No model to load")

    hist=model.fit_generator(gen(train_writers_mapping, batch_size=32), use_multiprocessing=True,
                        validation_data=gen(val_writers_mapping, batch_size=32), epochs=50, verbose=1,
                        workers=4, callbacks=callbacks_list, steps_per_epoch=200, validation_steps=100)


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, 3)]    0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, None, 100)    21600       input_2[0][0]                    
__________________________________________________________________________________________________
global_max_pooling1d_1 (GlobalM (None, 100)          0           model_2[1][0]                    
__________________________________________________________________________________________________
global_average_pooling1d_1 (Glo (None, 100)          0           model_2[1][0]                    
____________________________________________________________________________________________

In [ ]:
eval = model.evaluate_generator(gen(val_writers_mapping, batch_size=32), steps=100)
eval

NameError: ignored

In [ ]:
from matplotlib import pyplot as plt
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['SparseCategoricalAccuracy']
val_acc=hist.history['val_SparseCategoricalAccuracy']
xc=range(20)

In [ ]:
plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
print (plt.style.available) # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])

In [ ]:
plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('num of Epochs')
plt.ylabel('accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])